# Bengali.AI Competition - ResNeXt Training (Ensemble)

### Team MuchLearningSuchWow

This notebook contains code for training the ResNeXt network we used in our ensemble. It is connected to Weights and Biases in order to keep track of progress and performance.

## Imports

In [ ]:
import os

from tqdm.auto import tqdm
import cv2
import pandas as pd
import keras
import numpy as np
from sklearn.model_selection import train_test_split
from keras.callbacks import ReduceLROnPlateau
import psutil

from keras.layers import Conv2D, BatchNormalization, Activation, Add, MaxPool2D, Dense, \
    Dropout, GlobalAveragePooling2D, Concatenate, Input, Flatten, AveragePooling2D, Add
from keras import Model
from keras.regularizers import l2
    
import gc
import wandb
from wandb.keras import WandbCallback

## Filenames

In [ ]:
train_filename = 'input/bengaliai-cv19/train.csv'
model_filename = 'output/model_resnext.hdf5'

In [ ]:
if not os.path.isdir('output'):
    os.mkdir('output')

## Constants

In [ ]:
WEIGHT_DECAY = 5e-4

## Loading Dataframes

In [ ]:
train_df_ = pd.read_csv(train_filename)
train_df_ = train_df_.drop(['grapheme'], axis=1)

## Weights and Biases

In [ ]:
run = wandb.init(project='bengali')

In [ ]:
config = run.config
config.blocks = [
    {
        'width': 64,
        'output_width': 128,
        'cardinality': 24,
        'count': 2
    },
    {
        'width': 128,
        'output_width': 256,
        'cardinality': 24,
        'count': 3
    },
    {
        'width': 256,
        'output_width': 512,
        'cardinality': 24,
        'count': 2
    }
]
config.iChannels = 32
config.epochs = 120
config.max_lr = 0.0016
config.min_lr = 0.0004
config.n_cycles = 8
config.batch_size = 70
config.validation_split = 0.08
config.steps_per_epoch = int(200840*(1-config.validation_split))//config.batch_size//4

## Building ResNeXt Model

In [ ]:
def init_block(x, iChannels):
    x = Conv2D(iChannels, (7, 7), strides=2, padding='same', use_bias=False, kernel_initializer='he_normal',
               kernel_regularizer=l2(WEIGHT_DECAY))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size=(2, 2))(x)

    return x

In [ ]:
def resnext_block(x, width, output_width, cardinality):
    x = Conv2D(filters=width, padding='same', kernel_size=3)(x)
    inp = x
    inp = Conv2D(output_width, padding='same', kernel_size=1)(inp)
    subblocks = []

    for i in range(cardinality):
        y = Conv2D(filters=width, kernel_size=1)(x)
        # y = BatchNormalization()(y)#name=f'bn_1_{np.random.random()}')(y)
        y = Activation('relu')(y)
        y = Conv2D(filters=width, kernel_size=3, padding='same')(y)
        # y = BatchNormalization()(y)#name=f'bn_3_{np.random.random()}')(y)
        y = Activation('relu')(y)
        subblocks.append(y)

    x = Concatenate()(subblocks)
    x = Conv2D(output_width, kernel_size=1)(x)
    x = BatchNormalization()(x)
    x = Add()([x, inp])

    x = Activation('relu')(x)

    return x

In [ ]:
def resnext(blocks, iChannels, input_size=(64, 64, 1)):
    x = Input(shape=input_size)
    inp = x

    x = init_block(x, iChannels)

    for b in blocks:
        for i in range(b['count']):
            x = resnext_block(x, b['width'], b['output_width'], b['cardinality'])
        x = MaxPool2D()(x)

    x = GlobalAveragePooling2D()(x)

    x = Dense(2048, activation="relu")(x)
    x = Dropout(rate=0.12)(x)
    x = Dense(1024, activation="relu")(x)

    head_root = Dense(168, activation='softmax', name='dense_a')(x)
    head_vowel = Dense(11, activation='softmax', name='dense_b')(x)
    head_consonant = Dense(7, activation='softmax', name='dense_c')(x)

    model = Model(inputs=inp, outputs=[head_root, head_vowel, head_consonant])

    return model

In [ ]:
model = build_resnext(config.blocks, config.iChannels)

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              loss_weights=[2, 1, 1], 
              metrics=['accuracy', keras.metrics.Recall()])

## Training

In [ ]:
validation_steps = int(200840*config.validation_split)//config.batch_size//4

In [ ]:
class MultiOutputDataGenerator(keras.preprocessing.image.ImageDataGenerator):

    def flow(self,
             x,
             y=None,
             batch_size=32,
             shuffle=True,
             sample_weight=None,
             seed=None,
             save_to_dir=None,
             save_prefix='',
             save_format='png',
             subset=None):

        targets = None
        target_lengths = {}
        ordered_outputs = []
        for output, target in y.items():
            if targets is None:
                targets = target
            else:
                targets = np.concatenate((targets, target), axis=1)
            target_lengths[output] = target.shape[1]
            ordered_outputs.append(output)

        for flowx, flowy in super().flow(x, targets, batch_size=batch_size,
                                         shuffle=shuffle):
            target_dict = {}
            i = 0
            for output in ordered_outputs:
                target_length = target_lengths[output]
                target_dict[output] = flowy[:, i: i + target_length]
                i += target_length

            yield flowx, target_dict

In [ ]:
learning_rate_reduction_root = ReduceLROnPlateau(monitor='dense_a_accuracy',
                                                 patience=3,
                                                 verbose=1,
                                                 factor=0.5,
                                                 min_lr=0.00001)
learning_rate_reduction_vowel = ReduceLROnPlateau(monitor='dense_b_accuracy',
                                                  patience=3,
                                                  verbose=1,
                                                  factor=0.5,
                                                  min_lr=0.00001)
learning_rate_reduction_consonant = ReduceLROnPlateau(monitor='dense_c_accuracy',
                                                      patience=3,
                                                      verbose=1,
                                                      factor=0.5,
                                                      min_lr=0.00001)

In [ ]:
datagen = MultiOutputDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=8,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range=0.15,  # Randomly zoom image
            width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False,   # randomly flip images
            rescale=1.0/255.0,
            validation_split=config.validation_split)

train_df_['image_id'] = train_df_['image_id'].astype(str)+'.png'

Y_train_root = pd.get_dummies(train_df_.set_index('image_id')['grapheme_root'], dtype=np.float32)
Y_train_vowel = pd.get_dummies(train_df_.set_index('image_id')['vowel_diacritic'], dtype=np.float32)
Y_train_consonant = pd.get_dummies(train_df_.set_index('image_id')['consonant_diacritic'], dtype=np.float32)

def generator_wrapper(gen: MultiOutputDataGenerator, df: pd.DataFrame, subset: str, batch_size: int):
    for flowx, flowy in gen.flow_from_dataframe(df, 
                                                color_mode='grayscale', 
                                                directory='data', 
                                                x_col='image_id',
                                                y_col='image_id', 
                                                class_mode='raw', 
                                                target_size=(64, 64), 
                                                subset=subset, 
                                                batch_size=batch_size, 
                                                shuffle=True):
        yield flowx, {
            'dense_a': Y_train_root.loc[flowy].values,
            'dense_b': Y_train_vowel.loc[flowy].values,
            'dense_c': Y_train_consonant.loc[flowy].values,
        }

validation_generator = generator_wrapper(datagen, train_df_, 'validation', config.batch_size)
model.fit_generator(generator_wrapper(datagen, train_df_, 'training', config.batch_size), 
                    validation_data=validation_generator, 
                    validation_steps=validation_steps,
                    epochs=config.epochs, 
                    steps_per_epoch=config.steps_per_epoch,
                    callbacks=[learning_rate_reduction_root, 
                               learning_rate_reduction_vowel, 
                               learning_rate_reduction_consonant,
                               WandbCallback(data_type='image')])

## Saving Model

In [ ]:
model.save(os.path.join(wandb.run.dir, model_filename))